In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import datetime
from datetime import timedelta


from transplant.config import *
from transplant.data.dataset import Dataset


from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score

import random


%matplotlib inline

In [2]:
from transplant.data.validation_tree import *

In [3]:
validation = Validation()

In [4]:
#Test the validation tree for one instance

In [5]:
data_static = validation.get_static()
data_dynamic = validation.get_dynamic()

In [6]:
id_test = random.choice(data_static.id_patient.values)
id_test

179

In [7]:
validation.decision_empirique(id_patient=id_test, 
                              data_static=data_static, 
                              data_dynamic=data_dynamic,
                              regle_nan=True, 
                              drop_NMT=True)

'Transfer to ICU with mechanical ventilation and iNO therapy'

In [8]:
data_static_id=validation.get_static_id(data_static, id_=id_test)
data_static_id.immediate_extubation.values[0]

0

In [9]:
#apply the validation tree for all the id patients and get score performance

In [10]:
df_results = validation.get_results_dataframe(regle_nan=True, drop_NMT=True)

In [11]:
df_results.head()

,id_patient,immediate_extubation,target,result_tree,bianary_result_tree
0,1,0,0,Transfer to ICU with ECMO and mechanical venti...,0
1,2,1,1,Transfer to ICU with mechanical ventilation,0
2,3,0,0,Transfer to ICU with ECMO and mechanical venti...,0
3,4,0,1,Transfer to ICU with mechanical ventilation,0
4,5,0,0,Transfer to ICU with mechanical ventilation an...,0


In [12]:
df_results.result_tree.value_counts()

Transfer to ICU with ECMO and mechanical ventilation           142
Transfer to ICU with mechanical ventilation and iNO therapy     83
Transfer to ICU with standard oxygen therapy or NIV             59
Transfer to ICU with mechanical ventilation                     46
Name: result_tree, dtype: int64

In [13]:
#predictions of the tree vs 'immediate_extubation'

In [14]:
validation.get_score_and_confusion_matrices_for_immediate_ext(df_results)

accuracy:  0.693939393939394


matrice de confusion
[[193  23]
 [ 78  36]]


matrice de confusion en pct
[[58.48484848  6.96969697]
 [23.63636364 10.90909091]]


In [15]:
#predictions of the tree vs target

In [16]:
validation.get_score_and_confusion_matrices_for_target(df_results)

accuracy:  0.6121212121212121


matrice de confusion
[[166  23]
 [105  36]]


matrice de confusion en pct
[[50.3030303   6.96969697]
 [31.81818182 10.90909091]]


In [17]:
df_results_nan_false = validation.get_results_dataframe(regle_nan=False, drop_NMT=True)

In [18]:
validation.get_score_and_confusion_matrices_for_immediate_ext(df_results_nan_false)

accuracy:  0.693939393939394


matrice de confusion
[[193  23]
 [ 78  36]]


matrice de confusion en pct
[[58.48484848  6.96969697]
 [23.63636364 10.90909091]]


----------------------------------
<center> Regarder les patients qu'on aurait du extuber immédiatement d'après l'arbre mais qui ne l'ont pas été en réel </center>  
    
----------------------------------

In [19]:
list_ids = df_results_nan_false.id_patient[(df_results_nan_false['bianary_result_tree']==1) & (df_results_nan_false['immediate_extubation']==0)]

In [20]:
len(list_ids)

23

In [21]:
dataset = Dataset()

In [22]:
X,Y = dataset.get_static()

In [23]:
data_static = pd.concat([X, Y])

In [31]:
data_to_test = data_static[['id_patient','date_transplantation','heure_arrivee_bloc']][data_static.id_patient.isin(list_ids)]

In [32]:
data_to_test

,id_patient,date_transplantation,heure_arrivee_bloc
339,340,2017-06-17,09:21:00
408,410,2018-06-27,18:30:00
219,220,2015-09-08,00:15:00
17,18,2012-04-07,07:15:00
9,10,2012-02-09,14:00:00
356,357,2017-08-27,07:12:00
257,258,2016-03-28,13:15:00
216,217,2015-08-28,07:05:00
315,316,2017-02-17,10:30:00
124,125,2014-01-16,00:00:00


In [30]:
[x for x in data_static.columns if 'heure' in x]

['heure_arrivee_bloc']

In [33]:
df_results_nan_false[df_results_nan_false.id_patient.isin(list_ids)]

,id_patient,immediate_extubation,target,result_tree,bianary_result_tree
7,9,0,0,Transfer to ICU with standard oxygen therapy o...,1
8,10,0,0,Transfer to ICU with standard oxygen therapy o...,1
16,18,0,0,Transfer to ICU with standard oxygen therapy o...,1
21,23,0,0,Transfer to ICU with standard oxygen therapy o...,1
34,38,0,0,Transfer to ICU with standard oxygen therapy o...,1
35,39,0,0,Transfer to ICU with standard oxygen therapy o...,1
95,107,0,0,Transfer to ICU with standard oxygen therapy o...,1
97,109,0,0,Transfer to ICU with standard oxygen therapy o...,1
111,125,0,0,Transfer to ICU with standard oxygen therapy o...,1
119,133,0,1,Transfer to ICU with standard oxygen therapy o...,1


In [34]:
4/23

0.17391304347826086

----------------------------------
<center> Regarder les patients qui ont été extubés imm alors que non prévu par l'arbre </center>  
    
----------------------------------

In [56]:
data_static = validation.get_static()
data_dynamic = validation.get_dynamic()

In [36]:
list_ids = df_results_nan_false.id_patient[(df_results_nan_false['bianary_result_tree']==0) & (df_results_nan_false['immediate_extubation']==1)]

In [39]:
sub_df=df_results_nan_false[df_results_nan_false.id_patient.isin(list_ids)]

In [40]:
sub_df['result_tree'].value_counts()

Transfer to ICU with mechanical ventilation and iNO therapy    43
Transfer to ICU with mechanical ventilation                    20
Transfer to ICU with ECMO and mechanical ventilation           15
Name: result_tree, dtype: int64

Transfer to ICU with mechanical ventilation and iNO therapy

In [44]:
list_ino = sub_df.id_patient[sub_df.result_tree=='Transfer to ICU with mechanical ventilation and iNO therapy']

In [45]:
list_ino.shape

(43,)

In [69]:
count_test_augm_ratio_pao2_fio2 = 0
count_test_PAP_diminution = 0
count_both = 0

for id_patient in list_ino:
    data_static_id = validation.get_static_id(data_static, id_=id_patient)
    data_dynamic_id = validation.get_dynamic_id(data_dynamic, id_=id_patient)
    data_dynamic_id_end = validation.get_end_surgery_data_dynamic(data_dynamic_id,time=15, period=15)
    
    Test1 = validation.test_augm_ratio_pao2_fio2(data_static_id, data_dynamic_id_end, data_dynamic_id, threshold_augm=0, regle_nan=False)
    Test2 = validation.test_PAP_diminution(data_dynamic_id_end, paps_max=50, paps_min=32, regle_nan=False)
    
    if Test1 == False:
        if Test2==False:
            count_both = count_both+1
        else:
            count_test_augm_ratio_pao2_fio2=count_test_augm_ratio_pao2_fio2 + 1
    if Test2 == False and Test1 :
            count_test_PAP_diminution = count_test_PAP_diminution + 1

In [70]:
print("nombre de test 'augmentation du ratio Pa02/FiO2' negatif : ", count_test_augm_ratio_pao2_fio2)
print("nombre de test 'diminution des PAP' negatif : ", count_test_PAP_diminution)
print("nombre de test 'augmentation du ratio Pa02/FiO2' et 'diminution des PAP' negatif : ", count_both)

nombre de test 'augmentation du ratio Pa02/FiO2' negatif :  15
nombre de test 'diminution des PAP' negatif :  20
nombre de test 'augmentation du ratio Pa02/FiO2' et 'diminution des PAP' negatif :  8


Transfer to ICU with mechanical ventilation 

In [71]:
list_mech = sub_df.id_patient[sub_df.result_tree=='Transfer to ICU with mechanical ventilation']

In [76]:
list_mech.shape

(20,)

In [79]:
count_test_temp = 0
count_test_PASM = 0
count_test_noradrenaline = 0
count_test_ratio_pao2_fio2 = 0

for id_patient in list_mech:
    data_static_id = validation.get_static_id(data_static, id_=id_patient)
    data_dynamic_id = validation.get_dynamic_id(data_dynamic, id_=id_patient)
    data_dynamic_id_end = validation.get_end_surgery_data_dynamic(data_dynamic_id,time=15, period=15)
    
    Test1_1 = validation.test_temperature_sup36(data_dynamic_id_end,threshold=90, regle_nan=False)
    Test1_2 = validation.test_PASm_sup(data_dynamic_id_end, obj=60, threshold=85, regle_nan=False)
    Test1_3 = validation.test_noradrenaline(data_static_id, nora_max=1.5, nora_min=1,regle_nan=False)
    Test1_4 = validation.test_ratio_pao2_fio2(data_static_id, data_dynamic_id_end, obj=300, threshold=80, regle_nan=False)
    
    if Test1_1 == False:
        count_test_temp = count_test_temp + 1
    if Test1_2 == False:
        count_test_PASM = count_test_PASM + 1
    if Test1_3 == False:
        count_test_noradrenaline = count_test_noradrenaline + 1
    if Test1_4 == False:
        count_test_ratio_pao2_fio2 = count_test_ratio_pao2_fio2 + 1
        
    
print("nombre de test 'Temp > 36°' negatif : ", count_test_temp)
print("nombre de test 'PAsM > 60' negatif : ", count_test_PASM)
print("nombre de test 'noradrenaline < 1mg/h' negatif : ", count_test_noradrenaline)
print("nombre de test 'rapport Pa02/fi02 > 300' negatif : ", count_test_ratio_pao2_fio2)

nombre de test 'Temp > 36°' negatif :  8
nombre de test 'PAsM > 60' negatif :  3
nombre de test 'noradrenaline < 1mg/h' negatif :  7
nombre de test 'rapport Pa02/fi02 > 300' negatif :  13


Transfer to ICU with ECMO and mechanical ventilation

In [80]:
list_ecmo = sub_df.id_patient[sub_df.result_tree=='Transfer to ICU with ECMO and mechanical ventilation']

In [81]:
list_ecmo.shape

(15,)

In [82]:
count_test_PASM = 0
count_test_noradrenaline = 0
count_test_ratio_pao2_fio2_100 = 0
count_test_ratio_pao2_fio2_200 = 0
count_test_PAP_diminution = 0

for id_patient in list_ecmo:
    data_static_id = validation.get_static_id(data_static, id_=id_patient)
    data_dynamic_id = validation.get_dynamic_id(data_dynamic, id_=id_patient)
    data_dynamic_id_end = validation.get_end_surgery_data_dynamic(data_dynamic_id,time=15, period=15)
    
    Test1 = validation.test_PASm_sup(data_dynamic_id_end, obj=60, threshold=85, regle_nan=False)
    Test2 = validation.test_noradrenaline(data_static_id, nora_max=1.5, nora_min=1,regle_nan=False)
    Test3 = validation.test_ratio_pao2_fio2(data_static_id, data_dynamic_id_end, obj=200, threshold=80, regle_nan=False)
    Test4 = validation.test_ratio_pao2_fio2(data_static_id, data_dynamic_id_end, obj=100, threshold=80, regle_nan=False)
    Test5 = validation.test_PAP_diminution(data_dynamic_id_end, paps_max=50, paps_min=32, regle_nan=False)
    
    if Test1 == False:
        count_test_PASM = count_test_PASM + 1
    if Test2 == False:
        count_test_noradrenaline = count_test_noradrenaline + 1
    if Test3 == False:
        count_test_ratio_pao2_fio2_200 = count_test_ratio_pao2_fio2_200 + 1
    if Test4 == False:
        count_test_ratio_pao2_fio2_100 = count_test_ratio_pao2_fio2_100 + 1
    if Test5 == False : 
        count_test_PAP_diminution = count_test_PAP_diminution + 1
        
    
print("nombre de test 'PAsM > 60' negatif : ", count_test_PASM)
print("nombre de test 'noradrenaline < 1mg/h' negatif : ", count_test_noradrenaline)
print("nombre de test 'rapport Pa02/fi02 > 100' negatif : ", count_test_ratio_pao2_fio2_100)
print("nombre de test 'rapport Pa02/fi02 > 200' negatif : ", count_test_ratio_pao2_fio2_200)
print("nombre de test 'diminution des PAP' negatif : ", count_test_PAP_diminution)

nombre de test 'PAsM > 60' negatif :  3
nombre de test 'noradrenaline < 1mg/h' negatif :  2
nombre de test 'rapport Pa02/fi02 > 100' negatif :  1
nombre de test 'rapport Pa02/fi02 > 200' negatif :  6
nombre de test 'diminution des PAP' negatif :  6


In [ ]:
#lorsque les décisions ne matchent pas entre arbre et medecin, regarder si complications ou pas